In [ ]:
#pip install lyricsgenius

In [1]:
# Import dependencies
import lyricsgenius
from config import genius_token
import pandas as pd
import numpy as np
import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password
import time

In [2]:
genius = lyricsgenius.Genius(genius_token)

In [84]:
# Create DataFrame from Spotify API calls
song_genres_df = pd.read_csv('../Data/song_genres.csv')
song_genres_df.head()

,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['pop', 'viral', 'canadian', 'post-teen', 'dan...","['pop', 'viral', 'canadian', 'dance']",0.652,0.383,2,-7.076,0,0.0516,0.06760,0.00,0.0828,0.549,145.765,178994,4
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],0.889,0.340,11,-7.773,0,0.0697,0.21800,0.13,0.0550,0.716,94.009,174321,4
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"['pop', 'uk', 'dance']","['pop', 'uk', 'dance']",0.702,0.825,6,-3.787,0,0.0601,0.00883,0.00,0.0674,0.915,102.977,203064,4
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['pop', 'post-teen']",['pop'],0.736,0.802,0,-4.759,1,0.0864,0.46800,0.00,0.0940,0.675,144.005,172325,4
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['lgbtq+', 'pop', 'queer', 'country']","['pop', 'country']",0.810,0.511,5,-6.924,0,0.1640,0.12000,0.00,0.0832,0.837,151.947,154998,4


In [86]:
# Create a list of lyrics for each song
t0 = time.time()
lyrics_list = []
song_count = len(song_genres_df)
for i in range(int(song_count/100)+1):
    start = i*100
    end = (i+1)*100
    print(f'Finding lyrics for songs {start}-{end}:')
    for index, row in song_genres_df[start:end].iterrows():
        song = row['song']
        artist = row['artist']
        results = genius.search_songs(f'{song} {artist}')
        try:
            found_song = False
            for id, hit in enumerate(results['hits']):
                artist_name = hit['result']['primary_artist']['name'].lower()
                if row['artist'].lower() in artist_name:
                    found_song = True
                    song_id = hit['result']['id']
                    lyrics = genius.lyrics(urlthing=song_id)
                    lyrics_list.append(lyrics)
                    break
            if found_song == False:
                lyrics_list.append(None)
                print(index, 'Song not found.')
        except:
            lyrics_list.append(None)
            print(index, 'No result found.')
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'Lyrics added for {len(lyrics_list)} songs.')

Finding lyrics for songs 0-100:
2 Song not found.
25 Song not found.
34 Song not found.
40 Song not found.
41 Song not found.
44 Song not found.
50 Song not found.
52 Song not found.
54 Song not found.
57 Song not found.
58 Song not found.
59 Song not found.
60 Song not found.
61 Song not found.
63 Song not found.
64 Song not found.
66 Song not found.
78 Song not found.
79 Song not found.
87 Song not found.
88 Song not found.
89 Song not found.
90 Song not found.
91 Song not found.
93 Song not found.
96 Song not found.
97 Song not found.
98 Song not found.
99 Song not found.
Finding lyrics for songs 100-200:
100 Song not found.
101 Song not found.
102 Song not found.
103 Song not found.
104 Song not found.
105 Song not found.
106 Song not found.
107 Song not found.
108 Song not found.
109 Song not found.
110 Song not found.
111 Song not found.
112 Song not found.
113 Song not found.
114 Song not found.
115 Song not found.
116 Song not found.
117 Song not found.
118 Song not found.
119 

In [114]:
# Add a column of lyrics to the DataFrame
song_genres_df['lyrics'] = lyrics_list
song_genres_df.head()

,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,...,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['pop', 'viral', 'canadian', 'post-teen', 'dan...","['pop', 'viral', 'canadian', 'dance']",0.652,0.383,2,...,0,0.0516,0.06760,0.00,0.0828,0.549,145.765,178994,4,[Verse 1: Shawn Mendes]\nYou put me on a pedes...
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],0.889,0.340,11,...,0,0.0697,0.21800,0.13,0.0550,0.716,94.009,174321,4,"[Chorus]\nI'm not your friend\nOr anything, da..."
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"['pop', 'uk', 'dance']","['pop', 'uk', 'dance']",0.702,0.825,6,...,0,0.0601,0.00883,0.00,0.0674,0.915,102.977,203064,4,None
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['pop', 'post-teen']",['pop'],0.736,0.802,0,...,1,0.0864,0.46800,0.00,0.0940,0.675,144.005,172325,4,[Verse 1]\nHeaven sent you to me\nI'm just hop...
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['lgbtq+', 'pop', 'queer', 'country']","['pop', 'country']",0.810,0.511,5,...,0,0.1640,0.12000,0.00,0.0832,0.837,151.947,154998,4,"[Intro]\nT-T-Tay Keith, Took it to ten (Hey)\n..."


In [115]:
# Create a the clean_lyrics function
def clean_lyrics(song_genres_df):
    # Create non-destructive copy
    song_genres_df = pd.DataFrame(song_genres_df)
    
    # Create lists of symbols to clean
    regex = '\[(.*?)\]'
    symbols_to_space = ['\u205f', '\\u2005', '  ', '  ', '\n', '-', '–', '—', '_', '=']
    symbols_to_remove = [',', '.', '(', ')', "'", '’', '`', '[', ']', '{', '}', '"',
                         '”', '“', ':', ';', '*', '¨', '…', '‘', '·']
    other_symbols = ['?', '!'] 
    e = 'е'
    
    # Clean lyrics
    song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(regex, '', regex=True)
    for symbol in symbols_to_space:
        song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(symbol, ' ')
    for symbol in symbols_to_remove:
        song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(symbol, '')
    for symbol in other_symbols:
        song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(symbol, f' {symbol}')
    song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(e, 'e')
    
    return song_genres_df

In [116]:
# Run the clean_lyrics function
song_lyrics_df = clean_lyrics(song_genres_df)
song_lyrics_df.head()

,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,...,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['pop', 'viral', 'canadian', 'post-teen', 'dan...","['pop', 'viral', 'canadian', 'dance']",0.652,0.383,2,...,0,0.0516,0.06760,0.00,0.0828,0.549,145.765,178994,4,You put me on a pedestal and tell me Im the b...
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],0.889,0.340,11,...,0,0.0697,0.21800,0.13,0.0550,0.716,94.009,174321,4,Im not your friend Or anything damn You think...
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"['pop', 'uk', 'dance']","['pop', 'uk', 'dance']",0.702,0.825,6,...,0,0.0601,0.00883,0.00,0.0674,0.915,102.977,203064,4,None
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['pop', 'post-teen']",['pop'],0.736,0.802,0,...,1,0.0864,0.46800,0.00,0.0940,0.675,144.005,172325,4,Heaven sent you to me Im just hopin I dont re...
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['lgbtq+', 'pop', 'queer', 'country']","['pop', 'country']",0.810,0.511,5,...,0,0.1640,0.12000,0.00,0.0832,0.837,151.947,154998,4,T T Tay Keith Took it to ten Hey Ayy its a ...


In [117]:
# Count non alpha words
non_alpha_words = []
for index, row in song_lyrics_df.iterrows():
    filtered_words = row['lyrics']
    count = 0
    for word in list(set(str(filtered_words).strip().split(' '))):
        if not re.match(r'[a-zA-Z0-9?!/#$&+]+$', word):
            count+=1
    non_alpha_words.append(count)
print(f'{len(non_alpha_words)} non alphabetic words found.')

910 non alphabetic words found.


In [118]:
# Add a column for alpha word count
song_lyrics_df['non_alpha_words'] = non_alpha_words


In [119]:
## Print non alpha words for each song
#for item, row in song_lyrics_df[song_lyrics_df['non_alpha_words']>10].iterrows():
#    filtered_words = row['lyrics']
#    for word in list(set(str(filtered_words).strip().split(' '))):
#        if not re.match(r'[a-zA-Z0-9?!\/#$&+]+$', word):
#            print(row['song'], word)

In [120]:
# Remove songs with more than 10 non alpha words
song_lyrics_df = song_lyrics_df[song_lyrics_df['non_alpha_words']<=10]
song_lyrics_df.head()


,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,...,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['pop', 'viral', 'canadian', 'post-teen', 'dan...","['pop', 'viral', 'canadian', 'dance']",0.652,0.383,2,...,0.0516,0.06760,0.00,0.0828,0.549,145.765,178994,4,You put me on a pedestal and tell me Im the b...,1
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],0.889,0.340,11,...,0.0697,0.21800,0.13,0.0550,0.716,94.009,174321,4,Im not your friend Or anything damn You think...,1
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"['pop', 'uk', 'dance']","['pop', 'uk', 'dance']",0.702,0.825,6,...,0.0601,0.00883,0.00,0.0674,0.915,102.977,203064,4,None,0
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['pop', 'post-teen']",['pop'],0.736,0.802,0,...,0.0864,0.46800,0.00,0.0940,0.675,144.005,172325,4,Heaven sent you to me Im just hopin I dont re...,1
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['lgbtq+', 'pop', 'queer', 'country']","['pop', 'country']",0.810,0.511,5,...,0.1640,0.12000,0.00,0.0832,0.837,151.947,154998,4,T T Tay Keith Took it to ten Hey Ayy its a ...,1


In [121]:
# Find the number of songs without lyrics
song_lyrics_df['lyrics'].isnull().sum()

316

In [122]:
# Remove the songs without lyrics
song_lyrics_df = song_lyrics_df.dropna()
song_lyrics_df.head()

,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,...,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['pop', 'viral', 'canadian', 'post-teen', 'dan...","['pop', 'viral', 'canadian', 'dance']",0.652,0.383,2,...,0.0516,0.06760,0.00,0.0828,0.549,145.765,178994,4,You put me on a pedestal and tell me Im the b...,1
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],0.889,0.340,11,...,0.0697,0.21800,0.13,0.0550,0.716,94.009,174321,4,Im not your friend Or anything damn You think...,1
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['pop', 'post-teen']",['pop'],0.736,0.802,0,...,0.0864,0.46800,0.00,0.0940,0.675,144.005,172325,4,Heaven sent you to me Im just hopin I dont re...,1
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['lgbtq+', 'pop', 'queer', 'country']","['pop', 'country']",0.810,0.511,5,...,0.1640,0.12000,0.00,0.0832,0.837,151.947,154998,4,T T Tay Keith Took it to ten Hey Ayy its a ...,1
5,On Me,4Iedi94TIaB2GGb1nMB68v,Lil Baby,5f7VJjfbwm532GiveGC0ZK,toplists,"['atl', 'rap', 'trap']","['atl', 'rap', 'trap']",0.856,0.564,6,...,0.3920,0.00327,0.00,0.1340,0.483,77.972,135960,4,Whats happenin Chi Chi ? Fill the bando up ...,1


In [110]:
# Save DataFrame to CSV
song_lyrics_df.to_csv('../Data/song_lyrics.csv', index=False)


In [ ]:
# Add the clean movie function that takes in the argument, "movie"
def clean_movie(movie):
    # Create a non-destructive copy
    movie = dict(movie)
    alt_titles = {}
    # Combine alternate titles into one list
    for key in ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French',
               'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally',
               'Mandarin', 'McCune–Reischauer', 'Original title', 'Polish',
               'Revised Romanization', 'Romanized', 'Russian',
               'Simplified', 'Traditional', 'Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        
    # Merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')    
    return movie


In [ ]:
# Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames
    file_dir = '../Data/'
    kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}/ratings.csv')

    # Open and read the read the Wikipedia data JSON file
    with open('wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)    
    
    # Write a list comprehension to filter out TV shows
    wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie)
                  and ('imdb_link' in movie) 
                  and ('No. of episodes' not in movie)]

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # Read in the cleaned movies list from Step 4 as a DataFrame
    wiki_movies_df = pd.DataFrame(clean_movies)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)
    
    # Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame
    wiki_columns_to_keep = [col for col in wiki_movies_df.columns if wiki_movies_df[col].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # Create a variable that will hold the non-null values from the “Box office” column
    box_office = wiki_movies_df['Box office'].dropna()
    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Write a regular expression to match the six elements of "form_one" of the box office data
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    # Write a regular expression to match the three elements of "form_two" of the box office data
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?![\s[mb]illion])'

    # Add the parse_dollars function
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan
    
        # If input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            # convert to float and multiply by a million
            value = float(s) * 10**6
            # return value
            return value
    
        # If input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            # convert to float and multiply by a billion
            value = float(s) * 10**9
            # return value
            return value
    
        # If input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            # remove dollar sign and commas
            s = re.sub('\$|,', '', s)
            # convert to float
            value = float(s)
            # return value
            return value
    
        # Otherwise, return NaN
        else:
            return np.nan
    
    # Clean the box office column in the wiki_movies_df DataFrame
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # Clean the budget column in the wiki_movies_df DataFrame
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-–](?![a-z])', '$', regex=True)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    #wiki_movies_df.drop('Budget', axis=1, inplace=True)
    
    # Clean the release date column in the wiki_movies_df DataFrame
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|Feburary|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|Feburary|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)[0], infer_datetime_format=True)
    #wiki_movies_df.drop('Release date', axis=1, inplace=True)
    
    # Clean the running time column in the wiki_movies_df DataFrame
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s?ho?u?r?s?\s?(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    return wiki_movies_df, kaggle_metadata, ratings 


In [ ]:
# Create the path to your file directory and variables for the three files
file_dir = '../Data'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data
ratings_file = f'{file_dir}/ratings.csv'

In [ ]:
# Set the three variables equal to the function created in function_test.ipynb ?
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [ ]:
# Set the wiki_movies_df equal to the wiki_file variable
wiki_movies_df = wiki_file

In [ ]:
# Check that the wiki_movies_df DataFrame looks like this
wiki_movies_df.head()

In [ ]:
# Check that wiki_movies_df DataFrame columns are correct
wiki_movies_df.columns.to_list()